In [1]:
import pandas as pd
import pandas_datareader.data as pdr
from pandas import Series, DataFrame
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np

import datetime
from datetime import date, timedelta
import yfinance as yf
yf.pdr_override()

import plotly.graph_objects as go
from plotly.subplots import make_subplots

def fullPrint(df):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
        print(df)   


In [2]:
df = pdr.get_data_yahoo("TSLA", start="2016-01-01",end="2021-11-26")
sp = "^GSPC"
spdf = pdr.get_data_yahoo(sp, start="2016-01-01",end="2021-11-26")
sortLogic = {
    'Close': 'last',
    'Volume': 'sum'
}
dfSorted = df.resample("W-FRI").agg(sortLogic)
spDfSorted = spdf.resample("W-FRI").agg(sortLogic)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [3]:
weights = np.arange(1,31)
sumWeights = np.sum(weights)
dfSorted['30WMA'] = dfSorted['Close'].rolling(window=30).apply(lambda x: np.sum(weights*x)/sumWeights)

In [4]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=dfSorted.index, y=dfSorted['Close'],name='Price'),secondary_y=True)
fig.add_trace(go.Scatter(x=dfSorted.index, y=dfSorted['30WMA'],name="30WMA"),secondary_y=True)
fig.add_trace(go.Bar(x=dfSorted.index, y=dfSorted['Volume'],name="Volume"),secondary_y=False)
fig.show()

In [5]:
deltaX = 10.4
dfSorted['30WMASlope'] = dfSorted['30WMA'].diff()*500/(dfSorted['Close'].rolling(window=104).max()-dfSorted['Close'].rolling(window=104).min())/deltaX

In [6]:
dfSorted['P4WH'] = dfSorted['Close'].rolling(window=4).max()
dfSorted['P4WL'] = dfSorted['Close'].rolling(window=4).min()

In [7]:
dfSorted['VolumePerc'] = dfSorted['Volume'].pct_change()
def product(df,index):
    if index < 30:
        return 0
    else:
        return df.iloc[index].Close/df.iloc[index-30].Close
dfSorted['Percent'] = dfSorted.apply(lambda x: product(dfSorted,dfSorted.index.get_loc(x.name)),axis=1)
spDfSorted['Percent'] = dfSorted.apply(lambda x: product(spDfSorted,spDfSorted.index.get_loc(x.name)),axis=1)
dfSorted['RS'] = dfSorted['Percent'] - spDfSorted['Percent']
fullPrint(dfSorted['RS'])

Date
2016-01-08    0.000000
2016-01-15    0.000000
2016-01-22    0.000000
2016-01-29    0.000000
2016-02-05    0.000000
2016-02-12    0.000000
2016-02-19    0.000000
2016-02-26    0.000000
2016-03-04    0.000000
2016-03-11    0.000000
2016-03-18    0.000000
2016-03-25    0.000000
2016-04-01    0.000000
2016-04-08    0.000000
2016-04-15    0.000000
2016-04-22    0.000000
2016-04-29    0.000000
2016-05-06    0.000000
2016-05-13    0.000000
2016-05-20    0.000000
2016-05-27    0.000000
2016-06-03    0.000000
2016-06-10    0.000000
2016-06-17    0.000000
2016-06-24    0.000000
2016-07-01    0.000000
2016-07-08    0.000000
2016-07-15    0.000000
2016-07-22    0.000000
2016-07-29    0.000000
2016-08-05   -0.045521
2016-08-12   -0.060935
2016-08-19   -0.034409
2016-08-26    0.032652
2016-09-02    0.056826
2016-09-09    0.146488
2016-09-16    0.117606
2016-09-23   -0.021317
2016-09-30   -0.069268
2016-10-07   -0.117535
2016-10-14   -0.196359
2016-10-21   -0.173130
2016-10-28   -0.184213
2016-1

In [22]:
def checkIfStage1(price,volumePerc, RS, slope, WMA,P4WH,P4WL):
    stageOneIndicator = 0
    falseReason = ""
    if volumePerc<0.2:
        stageOneIndicator+=1
    else:
        falseReason += "volume "
    if slope<=0.5 and slope>=-0.3:
        stageOneIndicator+=1
    else:
        falseReason += "slope "
    if P4WH<P4WL*1.15:
        stageOneIndicator+=1
    else:
        falseReason += "range "
    if price <= WMA*1.1 and price>= WMA*0.9:
        stageOneIndicator+=1
    else:
        falseReason += "price "
    if stageOneIndicator>=3:
        return True
    return "False " + falseReason
def checkIfStage2(price,volumePerc, RS, slope, WMA,prevStage):
    if volumePerc < 0.2:
        if prevStage != "Stage 2":
            #print(prevStage)
            return "Volume"
    if RS < 0.1:
        return "RS"
    if slope < 0.5:
        return "Slope"
    if price < WMA*1.1:
        return "Price"
    return "Clear"
def checkStage(price,volumePerc, RS, slope, WMA,P4WH,P4WL,prevStage):
    stage1Check = checkIfStage1(price,volumePerc, RS, slope, WMA,P4WH,P4WL)
    stage2Check = checkIfStage2(price,volumePerc, RS, slope, WMA,prevStage)
    if stage1Check == True and stage2Check == "Clear":
        return "Error"
    if stage1Check == True:
        return "Stage 1 " + stage2Check 
    if stage2Check == "Clear":
        return "Stage 2"
    return stage2Check       

In [23]:
dfSorted = dfSorted.dropna()
dfSorted['Stage'] = ""
#dfSorted['Stage'] = dfSorted.apply(lambda x: checkStage(dfSorted.loc[index]['Close'],dfSorted.loc[index]['VolumePerc'],dfSorted.loc[index]['RS'],dfSorted.loc[index]['30WMASlope'],dfSorted.loc[index]['30WMA'],dfSorted.loc[index]['P4WH'],dfSorted.loc[index]['P4WL'],dfSorted.loc[index]['Stage'].shift()),axis=1)
for index, element in dfSorted.iterrows():
    if dfSorted.index.get_loc(index) == 0:
        continue
    dfSorted.iloc[dfSorted.index.get_loc(index), dfSorted.columns.get_loc('Stage')] = checkStage(dfSorted.loc[index]['Close'],dfSorted.loc[index]['VolumePerc'],dfSorted.loc[index]['RS'],dfSorted.loc[index]['30WMASlope'],dfSorted.loc[index]['30WMA'],dfSorted.loc[index]['P4WH'],dfSorted.loc[index]['P4WL'],dfSorted.iloc[dfSorted.index.get_loc(index) - 1]['Stage'])
    #dfSorted.loc[index]['Stage'] = checkStage(dfSorted.loc[index]['Close'],dfSorted.loc[index]['VolumePerc'],dfSorted.loc[index]['RS'],dfSorted.loc[index]['30WMASlope'],dfSorted.loc[index]['30WMA'],dfSorted.loc[index]['P4WH'],dfSorted.loc[index]['P4WL'],dfSorted.iloc[dfSorted.index.get_loc(index) - 1]['Stage'])
    #dfSorted.loc[index]['Stage'] = checkStage(dfSorted.loc[index]['Close'],dfSorted.loc[index]['VolumePerc'],dfSorted.loc[index]['RS'],dfSorted.loc[index]['30WMASlope'],dfSorted.loc[index]['30WMA'],dfSorted.loc[index]['P4WH'],dfSorted.loc[index]['P4WL'],dfSorted.shift(1).loc[index]['Stage'])
fullPrint(dfSorted['Stage'])

Date
2017-12-29                  
2018-01-05                RS
2018-01-12        Stage 1 RS
2018-01-19    Stage 1 Volume
2018-01-26    Stage 1 Volume
2018-02-02    Stage 1 Volume
2018-02-09                RS
2018-02-16    Stage 1 Volume
2018-02-23    Stage 1 Volume
2018-03-02        Stage 1 RS
2018-03-09    Stage 1 Volume
2018-03-16        Stage 1 RS
2018-03-23    Stage 1 Volume
2018-03-30                RS
2018-04-06                RS
2018-04-13    Stage 1 Volume
2018-04-20    Stage 1 Volume
2018-04-27    Stage 1 Volume
2018-05-04                RS
2018-05-11    Stage 1 Volume
2018-05-18            Volume
2018-05-25    Stage 1 Volume
2018-06-01    Stage 1 Volume
2018-06-08        Stage 1 RS
2018-06-15                RS
2018-06-22    Stage 1 Volume
2018-06-29    Stage 1 Volume
2018-07-06                RS
2018-07-13    Stage 1 Volume
2018-07-20    Stage 1 Volume
2018-07-27        Stage 1 RS
2018-08-03                RS
2018-08-10                RS
2018-08-17    Stage 1 Volume
2018-08-2